In [1]:
import pydicom
import numpy as np
from PIL import Image

In [2]:
def analyze_dicom_windowing(dicom_path):
    # Read the DICOM file
    ds = pydicom.dcmread(dicom_path)

    # Get original pixel data
    original_pixels = ds.pixel_array

    # Get window settings
    window_center = ds.WindowCenter
    window_width = ds.WindowWidth
    if isinstance(window_center, pydicom.multival.MultiValue):
        window_center = window_center[0]
    if isinstance(window_width, pydicom.multival.MultiValue):
        window_width = window_width[0]

    # Apply rescale slope and intercept
    rescaled_pixels = original_pixels * ds.RescaleSlope + ds.RescaleIntercept

    # Apply windowing
    def apply_window(image, center, width):
        img_min = center - width // 2
        img_max = center + width // 2
        windowed = np.clip(image, img_min, img_max)
        windowed = ((windowed - img_min) / (img_max - img_min) * 255).astype(np.uint8)
        return windowed

    windowed_pixels = apply_window(rescaled_pixels, window_center, window_width)

    # Convert to RGB
    rgb_image = Image.fromarray(windowed_pixels).convert('RGB')
    rgb_pixels = np.array(rgb_image)

    # Print statistics
    print(f"Original pixel value range: {original_pixels.min()} to {original_pixels.max()}")
    print(f"Rescaled pixel value range: {rescaled_pixels.min():.2f} to {rescaled_pixels.max():.2f}")
    print(f"Windowed pixel value range: {windowed_pixels.min()} to {windowed_pixels.max()}")
    print(f"RGB pixel value range: {rgb_pixels.min()} to {rgb_pixels.max()}")
    print(f"Window Center: {window_center}, Window Width: {window_width}")
    
    # Histogram of windowed pixels
    hist, bins = np.histogram(windowed_pixels, bins=256, range=(0, 255))
    print("Histogram of windowed pixels:")
    print(hist)

    return original_pixels, rescaled_pixels, windowed_pixels, rgb_pixels

In [3]:
# Usage
dicom_path = "C:/MyPythonProjects/XipeAI/xipeai-frontend/uploads/extracted/0066/unnamed_8/IM-0024-0001.dcm"
original, rescaled, windowed, rgb = analyze_dicom_windowing(dicom_path)

Original pixel value range: -2048 to 1331
Rescaled pixel value range: -2048.00 to 1331.00
Windowed pixel value range: 0 to 255
RGB pixel value range: 0 to 255
Window Center: 40, Window Width: 400
Histogram of windowed pixels:
[224230     90     46     89     60    123     85    133    164     77
    168     74    173    111    215    229    107    232    138    292
    108    261    294    151    309    155    302    146    323    302
    154    292    137    337    164    284    313    152    303    144
    309    134    275    271    140    265    111    257    121    240
    114    216    216     94    171    107    214     96    204    207
     98    176     88    169    100    193    155    105    196     82
    182     81    163    186     89    191     87    180     85    184
    180     97    195    100    180     85    163    208     98    205
    100    213    113    196    189    110    205    116    237     99
    239    119    264    247    106    241    113    250    114 